# Assignment 4 (solutions): Attention

### Due Date: Nov 22 (both sections)

### Grade (100 pts, 10%)

#### Your Name: Linpei Zhang

#### Your EID: lz497

*Note: This assignment covers material from the recording, notes, demo, and suggested readings from Lectures 5,8,9*

---

### Problem introduction

In this assignment we will explore the concept of attention using Word2Vec features. Recall that in lecture-05 we used the Word2Vec algorithm to compute feature representations of english words. For every center-word - context-word pair, ${\textbf{x}_w, \textbf{x}_c}$, in the training set, Word2Vec tries to maximize the inner product between their feature embeddings, $\textbf{u}_w$ and $\textbf{v}_c$, by first computing a vector of logits as $\textbf{z} = \textbf{u}_w \cdot \textbf{V}^T \in \mathbb{R}^N$, and then maximizing $p(\textbf{x}_c | \textbf{x}_w) = \sigma_{softmax}(\textbf{z})$ to learn $\textbf{U}$ and $\textbf{V}$ via MLE. 

Let's assume we are given a sequence of words:

$$X = \{ \textbf{x}^{(1)}, \textbf{x}^{(2)}, \textbf{x}^{(3)}, \textbf{x}^{(4)}, \dots, \textbf{x}^{(T)} \}$$

a corresponding sequence of center-word feature representations:

$$U = \{ \textbf{u}^{(1)}, \textbf{u}^{(2)}, \textbf{u}^{(3)}, \textbf{u}^{(4)}, \dots, \textbf{u}^{(T)} \}$$

and a corresponding context-word feature representations:

$$V = \{ \textbf{v}^{(1)}, \textbf{v}^{(2)}, \textbf{v}^{(3)}, \textbf{v}^{(4)}, \dots, \textbf{v}^{(T)} \}$$

Then, using the Word2Vec variable naming convention above, recall that in simple self attention (Lecture-09) we compute a set of *context vectors*:

$$C = \{ \textbf{c}^{(1)}, \textbf{c}^{(2)}, \textbf{c}^{(3)}, \textbf{c}^{(4)}, \dots, \textbf{c}^{(T)} \}$$ 

each of which is computed according to:

$$
\textbf{c}^{(t)} = \sum_{t'=1}^{T} \alpha_{t,t'} \; \textbf{u}^{(t)} \quad where \quad \alpha_{t,t'} = \frac{ e^{ \textbf{u}^{(t)} \cdot \textbf{v}^{(t')} } }{ \sum_{t''} e^{ \textbf{u}^{(t)} \cdot \textbf{v}^{(t'')} } } \\
$$

The attention weights make up the following (non-symmetric!) matrix:

$$
\textbf{A} = 
\left[ \begin{array}{ccc}
\alpha_{1,1} & \dots & \alpha_{1,T} \\
\vdots & \ddots & \vdots \\
\alpha_{T,1} & \dots & \alpha_{T,T}
\end{array}\right] \; \in [0,1]^{T \times T} \quad where \quad \sum_{t'} \alpha_{t,t'} = 1
$$

### Test sentence

In [1]:
sentence = "Thomas Jefferson was an American statesman and Founding Father who served as the third president of the United States"

### Load Word2Vec vocab, embedding matrices

In the `assignments/a4/` folder you will find three .tsv files: a vocabulary (`metadata.tsv`), center-word Word2Vec embeddings (`vectors_center.tsv`), and context-word Word2Vec embeddings (`vectors_context.tsv`). These are taking from the Lecture-05 Word2Vec demo. The code below loads the center and context word embeddings for the above test sentence into two numpy arrays: $U$, and $V$. 

In [2]:
import numpy as np

def load_matrix(fpath):
    matrix = []
    with open(fpath, 'r') as fd:
        tsv = fd.read()
    for line in tsv.split('\n'):
        row = []
        for value in line.split('\t'):
            row.append(float(value))
        matrix.append(row)
    return np.array(matrix)

def load_vocab(fpath) -> dict:
    with open(fpath, "r") as fd:
        tsv = fd.read()
    vocab = {}
    for line in tsv.split('\n'):
        vocab[line.strip()] = len(vocab)
    return vocab

vocab = load_vocab("metadata.tsv")
U_lookup = load_matrix("vectors_center.tsv").T
V_lookup = load_matrix("vectors_context.tsv").T

N = len(vocab)
D = len(U_lookup)

print(f"vocab size: {N}")
print(f"embed dim: {D}")

print(f"center embedding lookup matrix shape: {U_lookup.shape}")
print(f"context embedding lookup matrix shape: {V_lookup.shape}")

vocab size: 2321
embed dim: 15
center embedding lookup matrix shape: (15, 2321)
context embedding lookup matrix shape: (15, 2321)


In [3]:
word_list = sentence.lower().split(" ")
X = [vocab.get(word, N) for word in word_list]
print(f"Word indices: {X}")

Word indices: [2092, 1075, 2321, 2321, 79, 2321, 2321, 795, 2321, 2321, 2321, 2321, 2321, 2321, 1622, 2321, 2321, 2158, 1987]


In [4]:
# Assign OOV words low attention weights
u_oov = 0.1 * np.mean(U_lookup, axis=1, keepdims=True)
v_oov = 0.1 * np.mean(V_lookup, axis=1, keepdims=True) 
# Add the OOV embeddings to the lookup matrices
U_lookup = np.concatenate((U_lookup, u_oov), axis=1)
V_lookup = np.concatenate((V_lookup, v_oov), axis=1)
U_lookup.shape, V_lookup.shape

print(f"center word embedding lookup matrix w/oov embed added shape: {U_lookup.shape}")
print(f"context word embedding lookup matrix w/oov embed added shape: {U_lookup.shape}")

center word embedding lookup matrix w/oov embed added shape: (15, 2322)
context word embedding lookup matrix w/oov embed added shape: (15, 2322)


In [5]:
# Center-word embedding sequence of X
U = U_lookup[:, X]
# Context-word embedding sequence of X
V = V_lookup[:, X]

print(f"sequence of center-word embeddings shape: {U.shape}")
print(f"sequence of context-word embeddings shape: {U.shape}")

sequence of center-word embeddings shape: (15, 19)
sequence of context-word embeddings shape: (15, 19)


# Questions

### (40 pts) Q1: Attention weights

1. Compute the attention weight matrix $\textbf{A}$ for this test sentence using the definitions provided in the intro (and in Lecture-09 as needed).

*Hint: You can use the batched `softmax()` function provided below to compute A.*

In [6]:
def softmax(Z) -> np.ndarray:
    """
    Computes a softmax over each row of Z
    """
    Z_exp = np.exp(Z - np.max(Z, axis=1, keepdims=True))
    partition = np.sum(Z_exp, axis=1, keepdims=True)
    return Z_exp / partition

In [7]:
def compute_A():
    T= np.shape(U)[1]
    A= np.zeros((T, T))
    for t in range(T):
        for t_ in range(T):
            A[t,t_]= np.dot(U[:,t].T, V[:,t_])
    A= softmax(A.T)
    return A
A= compute_A()
A

array([[5.74356406e-01, 3.81443431e-01, 1.21851746e-04, 1.21851746e-04,
        8.73904800e-03, 1.21851746e-04, 1.21851746e-04, 1.39863417e-02,
        1.21851746e-04, 1.21851746e-04, 1.21851746e-04, 1.21851746e-04,
        1.21851746e-04, 1.21851746e-04, 3.62530237e-03, 1.21851746e-04,
        1.21851746e-04, 8.02952211e-03, 8.35772837e-03],
       [1.72014207e-01, 7.89592190e-01, 6.16304130e-04, 6.16304130e-04,
        1.57767780e-03, 6.16304130e-04, 6.16304130e-04, 2.15121405e-02,
        6.16304130e-04, 6.16304130e-04, 6.16304130e-04, 6.16304130e-04,
        6.16304130e-04, 6.16304130e-04, 3.96380535e-03, 6.16304130e-04,
        6.16304130e-04, 2.15085594e-03, 1.79347392e-03],
       [6.11989573e-02, 6.52114170e-02, 4.66123704e-02, 4.66123704e-02,
        6.27084094e-02, 4.66123704e-02, 4.66123704e-02, 6.43780645e-02,
        4.66123704e-02, 4.66123704e-02, 4.66123704e-02, 4.66123704e-02,
        4.66123704e-02, 4.66123704e-02, 6.25377866e-02, 4.66123704e-02,
        4.66123704e-02

### (40 pts) Q2: Attention block

1. Now compute the attention block $C$ using the attention weights that you've computed

2. Compute the L2 norm of each of the resultant context vectors. Do the same for the center-word embeddings in the preceeding layer. How do the magnitudes compare? Is there a logical explanation for this?

*Hint: You need to compute $\textbf{c}^{(t)} \; \forall t \in \{1, \dots, T\}$.*

In [11]:
def compute_C():
    C= np.zeros((np.shape(U)[0],np.shape(U)[1]))
    C= C.T
    T= np.shape(U)[1]
    for t in range(T):
        C[t]= sum(A[t,i] * U.T[i] for i in range(T))
    return C.T   
C= compute_C()
C_L2= np.linalg.norm(C.T, axis=1, keepdims=True)
U_L2= np.linalg.norm(U.T, axis=1, keepdims=True)
print(C_L2.T)
print(U_L2.T)

[[2.58233272 2.76029449 0.95049638 0.95049638 2.07611731 0.95049638
  0.95049638 2.20270947 0.95049638 0.95049638 0.95049638 0.95049638
  0.95049638 0.95049638 2.39554166 0.95049638 0.95049638 2.09189047
  2.08111205]]
[[2.72844333 2.9714337  0.18668972 0.18668972 2.28072887 0.18668972
  0.18668972 2.61942014 0.18668972 0.18668972 0.18668972 0.18668972
  0.18668972 0.18668972 2.97048526 0.18668972 0.18668972 2.53529085
  2.53219772]]


The original encoder U is encodes directly according to the input sequnence, while C_L2_form is computed by the weights of importance of the input information. When we use the L2 form, we lessen the differences between information and make the useless or unimportant information close to 0, which will be restrained effects in the preceeding layer. Change the C into the L2 form of it, which makes our computing later more easier and lessens the store space.

### (20 pts) Q3: Visualize the attention weights

Now that we've computed a simple self attention transformation of our Word2Vec sequence representation, it's time to visually examine what this attention mechanism affords us. First recognize that all values in $\textbf{A}$ are in the range $[0,1]$, and that each row is normalized. Therefore, we really just need to generate a heatmap of this matrix, keeping in mind that the matrix indices correspond directly to the positions in the sequence.

1. Generate the heatmap in the cell provided below

2. Briefly describe/explain what you see in the heatmap.

3. Why is our attention matrix not symmetric?

*Hint: You can generate a heatmap of the attention weights by calling `plot_attention_weights(A)`.*

In [89]:
import plotly.express as px
def plot_attention_weights(att_matrix):
    fig = px.imshow(A, x=word_list, y=word_list, width=20, height=20)
    fig.update_layout(width=800, height=800)
    print(sentence)
    fig.show()    
    fig.write_html("A.html")

In [90]:
plot_attention_weights(A)

Thomas Jefferson was an American statesman and Founding Father who served as the third president of the United States


In the heatmap, there are some facts I could conclude.
1. The color isn't symmetric in our plot which means our attention matrix is not symmetric.
2. The heatmp swap the order of words in the sentence.
3. this is not a 1-1 relationship but a 1 to many, meaning that an output word is affected by more than one input word (each one with different importance), meaning that an output word is affected by more than one input word.

Why not symmetric?  
Because when we compute a set of context vectors, the formula $$\textbf{c}^{(t)} = \sum_{t'=1}^{T} \alpha_{t,t'} \; \textbf{u}^{(t)} \quad$$ results the symmetric and we use the normalize calculating, the process also results the unsymmetric.